Import library

In [1]:
import pandas as pd
import os

In [19]:
path=os.getcwd()
folder="dataset"
file="all_data.csv"
data=os.path.join(folder,file)
print(data)

dataset\all_data.csv


Read the dataset

In [17]:
dataset=pd.read_csv(data)
print(dataset[:2])

  Export commodity(Tons) commodity_name  Billionaires per Million people  \
0                  58.6M   Maize (corn)                             2.42   
1                  58.6M   Maize (corn)                             2.42   

  Billionaire_Name RichestNetworld(Bn)  CrimeIndex  SafetyIndex  \
0        Elon Musk                $195        49.3         50.7   
1        Elon Musk                $195        49.3         50.7   

   Spend on Education % education.spend per student  \
0                 0.054                     $15,500   
1                 0.054                     $15,500   

  Gini coefficient(World_bank)%  ... Population.Rank  \
0                        41.30%  ...               3   
1                        41.30%  ...               3   

  GlobalPeaceIndex(GPI)2024(1-5)  Safety_index  GDP.Rank GDP(IMF'24)  \
0                          2.622          68.0         1   $28.78 Tn   
1                          2.622          68.0         1   $28.78 Tn   

   GDP(UN'22)  GD

In [125]:
print(dataset.columns)

Index(['Export commodity(Tons)', 'commodity_name',
       'Billionaires per Million people', 'Billionaire_Name',
       'RichestNetworld(Bn)', 'CrimeIndex', 'SafetyIndex',
       'Spend on Education %', 'education.spend per student',
       'Gini coefficient(World_bank)%', 'Gini coefficient(CIA)%', 'Area(km)',
       'Density', 'Population_Change%', 'Global_pop%', 'Population.Rank',
       'GlobalPeaceIndex(GPI)2024(1-5)', 'Safety_index', 'GDP.Rank',
       'GDP(IMF'24)', 'GDP(UN'22)', 'GDP PER CAPITAL', 'Population', 'Country',
       'Year'],
      dtype='object')


Statistic report

In [126]:
print(dataset.describe())

       Billionaires per Million people   CrimeIndex  SafetyIndex  \
count                      1651.000000  2980.000000  2980.000000   
mean                          1.063008    45.638490    54.361510   
std                           1.721108    14.419052    14.419052   
min                           0.006000    15.600000    18.800000   
25%                           0.144000    33.700000    45.000000   
50%                           0.509000    46.400000    53.600000   
75%                           1.093000    55.000000    66.300000   
max                          10.892000    81.200000    84.400000   

       Spend on Education %      Density  Population_Change%  Population.Rank  \
count           3706.000000  3784.000000         3784.000000      3784.000000   
mean               0.042636   203.559461            0.012593        88.090645   
std                0.017621   609.355444            0.009086        54.585284   
min                0.002000     2.000000            0.000300   

Features -- Input columns

In [198]:
selected=["CrimeIndex","Spend on Education %","GlobalPeaceIndex(GPI)2024(1-5)","GDP(IMF'24)","GDP PER CAPITAL"]
x=dataset[selected].copy()
print(x[:2])

   CrimeIndex  Spend on Education %  GlobalPeaceIndex(GPI)2024(1-5)  \
0        49.3                 0.054                           2.622   
1        49.3                 0.054                           2.622   

  GDP(IMF'24) GDP PER CAPITAL  
0   $28.78 Tn         $82,877  
1   $28.78 Tn         $82,877  


Remove noisy values

In [199]:
def convert_gdp(val):
    val=str(val).replace("$","").strip()
    if "Tn" in val:
        return float(val.replace("Tn",""))*1e12
    elif "Bn" in val:
        return float(val.replace("Bn",""))*1e9
    elif "Mn" in val:
        return float(val.replace("Mn",""))*1e6
    else:
        return float(val)

In [200]:
x["GDP(IMF'24)"]=x["GDP(IMF'24)"].apply(convert_gdp)

In [201]:
print(x["GDP(IMF'24)"][:3])

0    2.878000e+13
1    2.878000e+13
2    2.878000e+13
Name: GDP(IMF'24), dtype: float64


In [202]:
x["GDP PER CAPITAL"]=x["GDP PER CAPITAL"].str.replace("$","",regex=False).str.replace(",","",regex=False)
print(x["GDP PER CAPITAL"][:3])

0    82877
1    82877
2    82877
Name: GDP PER CAPITAL, dtype: object


In [203]:
x["GDP PER CAPITAL"]=pd.to_numeric(x["GDP PER CAPITAL"])

Null values check

In [204]:
print(x.isnull().sum())

CrimeIndex                        804
Spend on Education %               78
GlobalPeaceIndex(GPI)2024(1-5)    311
GDP(IMF'24)                         0
GDP PER CAPITAL                     0
dtype: int64


Imputer to fill null values

In [208]:
from sklearn.impute import SimpleImputer

median=SimpleImputer(strategy="median")
mean=SimpleImputer(strategy="mean")

In [209]:

x["CrimeIndex"]=median.fit_transform(x[["CrimeIndex"]])
x[["Spend on Education %","GlobalPeaceIndex(GPI)2024(1-5)"]]=mean.fit_transform(x[["Spend on Education %",
                                                                                       "GlobalPeaceIndex(GPI)2024(1-5)"]])

In [210]:
print(x.isnull().sum())

CrimeIndex                        0
Spend on Education %              0
GlobalPeaceIndex(GPI)2024(1-5)    0
GDP(IMF'24)                       0
GDP PER CAPITAL                   0
dtype: int64


Target --Output values

In [ ]:
y=dataset["Gini coefficient(World_bank)%"].copy()
print(y.head())

0    41.30%
1    41.30%
2    41.30%
3    41.30%
4    41.30%
Name: Gini coefficient(World_bank)%, dtype: object


Remove noisy

In [213]:
y=y.str.replace("%","")
print(y.head())

0    41.30
1    41.30
2    41.30
3    41.30
4    41.30
Name: Gini coefficient(World_bank)%, dtype: object


In [214]:
y=pd.to_numeric(y,errors="coerce")

Check null

In [215]:
print(y.isnull().sum())

192


Fill null values

In [216]:
y=y.fillna(y.median())
print(y.isnull().sum())

0


Split data to Train and Test

In [217]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

In [218]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(2838, 5)
(2838,)
(946, 5)
(946,)


Algorithms

In [219]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [247]:
model1=DecisionTreeRegressor(max_depth=5,random_state=0)
model2=RandomForestRegressor(n_estimators=200,max_depth=3,random_state=0)
model3=XGBRegressor(n_estimators=200,max_depth=5,random_state=0,learning_rate=0.1)

cross validation in MSE

In [277]:
from sklearn.model_selection import cross_val_score

cv_error1=cross_val_score(model1,x_train,y_train,cv=5,scoring="neg_mean_squared_error")
cv_error2=cross_val_score(model2,x_train,y_train,cv=5,scoring="neg_mean_squared_error")
cv_error3=cross_val_score(model3,x_train,y_train,cv=5,scoring="neg_mean_squared_error")

In [279]:
print(f"DecisionTree Error: {-cv_error1} : Avg {np.mean(-cv_error1)}")
print(f"RandomForest Error: {-cv_error2} : Avg {np.mean(-cv_error2)}")
print(f"Xgboost      Error: {-cv_error3} : Avg {np.mean(-cv_error3)}")

DecisionTree Error: [19.22621599 23.58862937 18.16348854 24.37878232 23.40703222] : Avg 21.752829685699762
RandomForest Error: [21.49399833 27.74973561 24.69658962 30.00314369 29.03620852] : Avg 26.595935152641225
Xgboost      Error: [0.02645877 0.02893985 0.03163047 0.03179559 0.03304939] : Avg 0.03037481234555845


Cross validation in r2_score

In [281]:
cv_score1=cross_val_score(model1,x_train,y_train,cv=5,scoring="r2")
cv_score2=cross_val_score(model2,x_train,y_train,cv=5,scoring="r2")
cv_score3=cross_val_score(model3,x_train,y_train,cv=5,scoring="r2")

In [282]:
print(f"DecisionTree R2 score:{cv_score1} Avg {np.mean(cv_score1)}")
print(f"RandomForest R2 score:{cv_score2} Avg {np.mean(cv_score2)}")
print(f"Xgboost      R2 score:{cv_score3} Avg {np.mean(cv_score3)}")

DecisionTree R2 score:[0.58642131 0.56720884 0.64961425 0.58483894 0.57354293] Avg 0.5923252534739192
RandomForest R2 score:[0.53763862 0.49086316 0.52358639 0.48905828 0.47098393] Avg 0.502426075045137
Xgboost      R2 score:[0.99943084 0.99946903 0.99938983 0.99945853 0.99939787] Avg 0.999429219231527


Model train

In [286]:
model3.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=0, ...)

Model save

In [287]:
import pickle
with open("Trained_model.pkl","wb") as f:
    pickle.dump(model3,f)

Prediction

In [288]:
y_pred=model3.predict(x_test)

Test Mean_Squared_Error and R2_score

In [289]:
from sklearn.metrics import mean_squared_error,r2_score
mse=mean_squared_error(y_test,y_pred)
print(f"xgboost MSE: {mse}")

xgboost MSE: 0.02419725614037578


In [290]:
score=r2_score(y_test,y_pred)
print(f"Xgboost r2_score: {score}")

Xgboost r2_score: 0.9995629665896157


New data prediction

In [312]:
new_data=[[49.3,0.054,2.622,2.878000e+13,82877]]  
df=pd.DataFrame(new_data,columns=x.columns)
r=model3.predict(df)

Classify in the range of inequality %

In [313]:
if r<=25:
    print(f"{r} : Very Low Inequality")
elif r<=30:
    print(f"{r} : Low Inequality")
elif r<=40:
    print(f"{r} : Moderate Inequality") 
elif r<=45:
    print(f"{r} : High Inequality")
elif r<=50:
    print(f"{r} : Very high Inequality")
else:
    print(f"{r} : Extreme Inequality")        
               

[41.20891] : High Inequality
